In [116]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

max_workers = 10
max_editors = 100
email = 'd.ecer@elifesciences.org'

In [117]:
import logging
from datetime import datetime
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from typing import List

import pandas as pd
from tqdm.auto import tqdm

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import
import data_science_pipeline.configure_notebook_logging  # pylint: disable=unused-import

from data_science_pipeline.utils.europepmc import (
    EuropePMCApi,
    europepmc_requests_retry_session
)
from data_science_pipeline.utils.bq import to_gbq, is_bq_not_found_exception
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)

In [118]:
LOGGER = logging.getLogger(__name__)

logging.basicConfig(level='INFO')
logging.getLogger('data_science_pipeline').setLevel(logging.INFO)

In [119]:
editor_parsed_pubmed_links_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_links'
)

editor_parsed_pubmed_ids_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_ids'
)

In [120]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [121]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [134]:
_sql = (
    '''
    SELECT 
        person_id,
        pubmed_url,
        provenance.imported_timestamp as imported_timestamp
    FROM `{table}`
    ''').format(table=editor_parsed_pubmed_ids_table_name)

try:
    existing_editor_ids_and_pubmed_url_df = read_big_query(_sql)
except Exception as e:  # pylint: disable=broad-except
    if not is_bq_not_found_exception(e):
        raise
    print('table not found: %s' % editor_parsed_pubmed_ids_table_name)
    existing_editor_ids_and_pubmed_url_df = pd.DataFrame(columns=['person_id','pubmed_url', 'imported_timestamp'])
print(len(existing_editor_ids_and_pubmed_url_df))
existing_editor_ids_and_pubmed_url_df.head(3)

> ```sql
> 
>     SELECT 
>         person_id,
>         pubmed_url,
>         provenance.imported_timestamp as imported_timestamp
>     FROM `de_dev.data_science_editor_pubmed_ids`
>     where name in ("Eduardo Franco", "Andrew King", "Lu Chen")
>     
> ```

6


/Users/hazal/elife_repo/data-science-dags/venv/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1952: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


,person_id,pubmed_url,imported_timestamp
0,14601,http://www.ncbi.nlm.nih.gov/pubmed?term=King%2...,2021-08-20 15:25:00.233909+00:00
1,1086,https://tinyurl.com/PubMedEduardoFrancoooo,2021-08-20 15:25:00.233909+00:00
2,90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,2021-09-14 13:58:03.799614+00:00


In [123]:
existing_editor_ids_set = set(existing_editor_ids_and_pubmed_url_df['person_id'])
len(existing_editor_ids_set)

773

In [124]:
existing_editor_pubmed_url_set = set(existing_editor_ids_and_pubmed_url_df['pubmed_url'])
len(existing_editor_pubmed_url_set)

781

In [125]:
editor_parsed_pubmed_links_df = read_big_query(
    'SELECT * FROM `{table_name}`\nWHERE parsed_search_term IS NOT NULL'.format(
        table_name=editor_parsed_pubmed_links_table_name
    )
)
print(len(editor_parsed_pubmed_links_df))
editor_parsed_pubmed_links_df.head(3)

> ```sql
> SELECT * FROM `de_dev.data_science_editor_pubmed_links`
> WHERE parsed_search_term IS NOT NULL
> ```

763


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term
0,False,Barnabas Daru,"{'exclude': None, 'include': {'author': ['Daru...",187442,https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,"[32355257, 29083043, 30455213, 28919204]","[https://www.ncbi.nlm.nih.gov/pubmed/32355257,...",https://pubmed.ncbi.nlm.nih.gov/?term=Daru+BH&...,Daru BH
1,False,Yuuki Watanabe,"{'exclude': None, 'include': {'author': ['Wata...",126873,https://www.ncbi.nlm.nih.gov/pubmed/?term=Wata...,"[25902489, 23341596, 20946384]","[https://www.ncbi.nlm.nih.gov/pubmed/25902489,...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Wata...,Watanabe YY[Author]
2,False,Safi Darden,"{'exclude': None, 'include': {'author': ['Dard...",43038,https://pubmed.ncbi.nlm.nih.gov/?size=200&term...,"[32900316, 32531279, 31748621, 30800389]","[https://www.ncbi.nlm.nih.gov/pubmed/32900316,...",https://pubmed.ncbi.nlm.nih.gov/?size=200&term...,Darden SK


In [126]:
remaining_editor_parsed_pubmed_links_df = editor_parsed_pubmed_links_df[
    ~editor_parsed_pubmed_links_df['person_id'].isin(existing_editor_ids_set)
]

remaining_editor_parsed_pubmed_links_df = remaining_editor_parsed_pubmed_links_df.append(
    editor_parsed_pubmed_links_df[
        ~editor_parsed_pubmed_links_df['pubmed_url'].isin(existing_editor_pubmed_url_set)
    ], ignore_index=True
)

print(len(remaining_editor_parsed_pubmed_links_df))
remaining_editor_parsed_pubmed_links_df.head(3)

4


,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term
0,False,Lu Chen,"{'exclude': None, 'include': {'author': ['Chen...",90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,"[30782829, 30355624, 30068571, 29760176, 25843...","[https://pubmed.ncbi.nlm.nih.gov/30782829, htt...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,Chen L[Author]
1,False,Andrew King,"{'exclude': None, 'include': {'author': ['King...",14601,http://www.ncbi.nlm.nih.gov/pubmed?term=King%2...,"[31949136, 31300665, 29136122, 31802997, 28891...","[https://www.ncbi.nlm.nih.gov/pubmed/31949136,...",https://www.ncbi.nlm.nih.gov/pubmed?term=King%...,King A J[au]
2,False,Lu Chen,"{'exclude': None, 'include': {'author': ['Chen...",90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,"[30782829, 30355624, 30068571, 29760176, 25843...","[https://pubmed.ncbi.nlm.nih.gov/30782829, htt...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,Chen L[Author]


In [127]:
processing_editor_parsed_pubmed_links_df = remaining_editor_parsed_pubmed_links_df
if max_editors:
    processing_editor_parsed_pubmed_links_df = processing_editor_parsed_pubmed_links_df[:max_editors]
len(processing_editor_parsed_pubmed_links_df)

4

In [128]:
def get_editor_pubmed_paper_ids(europepmc_api: EuropePMCApi, row) -> List[str]:
    parsed_search_term = row.parsed_search_term
    if not parsed_search_term:
        return None
    author_names = parsed_search_term.get('include', {}).get('author')
    try:
        author_names = parsed_search_term.get('include', {}).get('author')
        return europepmc_api.get_author_pmids(author_names)
    except:  # pylint: disable=bare-except
        LOGGER.error('failed to retrieve pubmed ids for author names: %s', author_names, exc_info=1)
        return None


editor_pubmed_links_result_df = processing_editor_parsed_pubmed_links_df[:max_editors].copy()

with europepmc_requests_retry_session() as session:
    europepmc_api = EuropePMCApi(
        session,
        params={'email': email}
    )
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        editor_pubmed_links_result_df['pubmed_ids'] = list(tqdm(
            executor.map(
                lambda row: get_editor_pubmed_paper_ids(europepmc_api, row),
                editor_pubmed_links_result_df.itertuples()
            ),
            total=len(editor_pubmed_links_result_df),
            leave=False
        ))

In [131]:
non_empty_editor_pubmed_links_result_df = editor_pubmed_links_result_df[
    ~pd.isnull(editor_pubmed_links_result_df['pubmed_ids'])
].copy()
len(non_empty_editor_pubmed_links_result_df)
non_empty_editor_pubmed_links_result_df.head(3)

,is_ncbi_bibliography_url,name,parsed_search_term,person_id,pubmed_url,relevant_pubmed_ids,relevant_pubmed_urls,resolved_pubmed_url,search_term,pubmed_ids
0,False,Lu Chen,"{'exclude': None, 'include': {'author': ['Chen...",90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,"[30782829, 30355624, 30068571, 29760176, 25843...","[https://pubmed.ncbi.nlm.nih.gov/30782829, htt...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,Chen L[Author],"[34378575, 34374399, 34342306, 34323898, 34328..."
1,False,Andrew King,"{'exclude': None, 'include': {'author': ['King...",14601,http://www.ncbi.nlm.nih.gov/pubmed?term=King%2...,"[31949136, 31300665, 29136122, 31802997, 28891...","[https://www.ncbi.nlm.nih.gov/pubmed/31949136,...",https://www.ncbi.nlm.nih.gov/pubmed?term=King%...,King A J[au],"[34334720, 34351722, 34345801, 34290235, 34351..."
2,False,Lu Chen,"{'exclude': None, 'include': {'author': ['Chen...",90633,https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,"[30782829, 30355624, 30068571, 29760176, 25843...","[https://pubmed.ncbi.nlm.nih.gov/30782829, htt...",https://www.ncbi.nlm.nih.gov/pubmed/?term=Chen...,Chen L[Author],"[34378575, 34374399, 34342306, 34323898, 34328..."


In [132]:
non_empty_editor_pubmed_links_result_df['provenance'] = [{
    'source': 'europepmc',
    'imported_timestamp': datetime.utcnow().isoformat()
}] * len(non_empty_editor_pubmed_links_result_df)

In [133]:
if len(non_empty_editor_pubmed_links_result_df) == 0:
    print('no data to upload')
else:
    print('writing to:', editor_parsed_pubmed_ids_table_name)
    to_gbq(
        non_empty_editor_pubmed_links_result_df,
        project_id=project_id,
        destination_table=editor_parsed_pubmed_ids_table_name,
        if_exists='append'
    )
    print('done')

writing to: de_dev.data_science_editor_pubmed_ids


INFO:root:Processed 4 lines
INFO:data_science_pipeline.utils.bq:loading from /var/folders/gz/3b4st4q56pd1983txrbt2yyw0000gn/T/tmp410cpcr0/data.jsonl.gz
INFO:data_science_pipeline.utils.bq:File size is : 447932
INFO:data_science_pipeline.utils.bq:Loaded 4 rows into de_dev:data_science_editor_pubmed_ids.


done
